In [330]:
import requests
from bs4 import BeautifulSoup

In [331]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
}
url = "https://www.melon.com/chart/index.htm"

res = requests.get(url, headers=headers)
res

<Response [200]>

In [332]:
html = res.text
html

'<!DOCTYPE html>\r\n<html lang="ko">\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<head>\r\n\t\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\r\n\t\r\n\r\n\t\r\n\r\n\t\r\n\r\n\t<title>멜론차트>TOP100>멜론</title>\r\n\t<meta name="keywords" content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" />\r\n\t<meta name="description" content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" />\r\n\t<meta name="naver-site-verification" content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" />\r\n\t<meta name="google-site-verification" content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" />\r\n\t<meta property="fb:app_id" content="357952407588971"/>\r\n\t<meta property="og:title" content="Melon"/>\r\n\t<meta property="og:image" content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png"/>\r\n\t

In [333]:
import re
import pandas as pd
def 멜론차트_가져오기():
    
    soup = BeautifulSoup(html, "html.parser")


    song_list_items = soup.select("div", class_="service_list_song type02 d_song_list") # 전체 노래 리스트 부분
 
    song_title = []
    song_title_list = [] # 노래 제목 
    song_rank = []
    song_rank_list = [] # 순위
    album_list = [] # 앨범 명
    singer_list = [] # 가수 이름

    for song_list_item in song_list_items: # 전체 노래 리스트 순회
        
        wrap_song_info = song_list_item.find("div", class_="wrap_song_info") # 곡 정보(노래 제목, 가수 이름) 부분

        rank_info = song_list_item.find("span", class_="rank") 

        url_info = song_list_item.find("div", class_="wrap") 
        if rank_info:
            song_rank.append(rank_info.text)

        if wrap_song_info:

            ellipsis_rank01 = wrap_song_info.find("div", class_="ellipsis rank01") # 노래 제목 속성의 부모 class
            ellipsis_rank02 = wrap_song_info.find("div", class_="ellipsis rank02") # 가수 이름 속성의 부모 class
            ellipsis_rank03 = wrap_song_info.find("div", class_="ellipsis rank03") # 앨범 명 속성의 부모 class

            if ellipsis_rank01:
                song_title.append(ellipsis_rank01.text.split('\n')[1]) # 순위 정보가 \n과 섞여 있기에 \n 기준으로 split하여 가공
                song_url = ellipsis_rank01.find("a")["href"] # 커버 이미지 주소

            if ellipsis_rank02:
                singer_name = ellipsis_rank02.find("span").text.strip() # 가수 이름 정보를 담고있는 tag-span에 종종 공백을 담고 있기에 
                                                                        # 공백을 제외하고 담기
                singer_list.append(singer_name)
            if ellipsis_rank03:
                album_list.append(ellipsis_rank03.text.split('\n')[1])   

            wrap_song_info = ""

    song_title_list = list(dict.fromkeys(song_title))
    singer_list = singer_list[6:]

    for i in range(len(song_rank)):
        
        if song_rank[i] != "순위":
            song_rank_list.append(song_rank[i])
    #json 주소
    j_url = "https://www.melon.com/commonlike/getSongLike.json?contsIds=36599950%2C36617841%2C36635522%2C36430773%2C36382580%2C36356993%2C36411344%2C36490426%2C36416114%2C36599949%2C34061322%2C36411342%2C36318125%2C35454426%2C36502910%2C34908740%2C35931532%2C35008524%2C35945927%2C36632907%2C36546894%2C35008525%2C32508053%2C35834583%2C35834584%2C36391236%2C35008527%2C36581408%2C36546895%2C35008534%2C35985167%2C35008528%2C35008526%2C35454425%2C35008531%2C32872978%2C35008532%2C36331121%2C33507137%2C36313117%2C35008530%2C34819473%2C35546497%2C35640751%2C30244931%2C36516320%2C8037843%2C36635524%2C35849863%2C36351997%2C36206259%2C36546900%2C33666269%2C34847378%2C36110996%2C36546908%2C36180700%2C36601157%2C36105548%2C36334401%2C36546897%2C36404853%2C35729104%2C36441862%2C36241824%2C36001290%2C36430774%2C36546902%2C36604256%2C34657844%2C4446485%2C36546899%2C36546903%2C36518341%2C36546905%2C36635523%2C36546906%2C36440243%2C35299693%2C33658563%2C36428682%2C36365073%2C34864406%2C34431086%2C35667692%2C36394038%2C34754292%2C36322781%2C36480197%2C35535827%2C35252996%2C35730562%2C36637828%2C36500102%2C36616378%2C33496587%2C36359275%2C36416116%2C35643794%2C36595401"
    response = requests.get(j_url, headers=headers)
    data = response.json()
    like_check = [] # 좋아요 수
    id_check = [] # 곡 일련번호
    # 'CONTSID' 키가 있는 데이터만 추출
    contsid_data = [item for item in data['contsLike'] if 'SUMMCNT' in item]

    for i in range(len(contsid_data)):
        like_check.append(contsid_data[i]['SUMMCNT'])
        id_check.append(contsid_data[i]['CONTSID'])
    url = "https://www.melon.com/chart/index.htm"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    
    tbody_tag = soup.find("tbody")  # 커버 이미지 주소는 'tbody' tag -> 'img' 속성 -> 'src' 속성 안에 있음

    img_tags = tbody_tag.find_all("img")

    url_list = [img.get("src") for img in img_tags]
    
    melon_list = []

    for i in range(100): #  각 종류별 컬럼마다 data는 다르지만 길이는 같기에 무작위 하나의 컬럼의 길이 만큼 반복하여
                                    # 딕셔너리에 값 삽입
        id_res = id_check[i]               
        rank_res = song_rank_list[i]
        album_res = album_list[i] 
        title_res = song_title_list[i]
        name_res = singer_list[i]
        url_res = url_list[i]
        like_res = like_check[i]
        song_info = {               # 딕셔너리 구조
                "곡일련번호": id_res,
                "순위": rank_res,
                "앨범": album_res,
                "곡명": title_res,
                "가수": name_res,
                "커버이미지_주소" : url_res,
                "좋아요" : like_res,
            }
        melon_list.append(song_info)
    result = pd.DataFrame(melon_list) # 시각화 자료 return
 
    return result

    

In [334]:
melon_list

[{'곡일련번호': 36599950,
  '순위': '1',
  '앨범': "NewJeans 2nd EP 'Get Up'",
  '곡명': 'Super Shy',
  '가수': 'NewJeans',
  '커버이미지_주소': 'https://cdnimg.melon.co.kr/cm2/album/images/112/81/456/11281456_20230706180841_500.jpg/melon/resize/120/quality/80/optimize',
  '좋아요': 102339},
 {'곡일련번호': 36617841,
  '순위': '2',
  '앨범': 'Seven (feat. Latto) - Clean Ver.',
  '곡명': 'Seven (feat. Latto) - Clean Ver.',
  '가수': '정국',
  '커버이미지_주소': 'https://cdnimg.melon.co.kr/cm2/album/images/112/86/070/11286070_20230713181059_500.jpg/melon/resize/120/quality/80/optimize',
  '좋아요': 106400},
 {'곡일련번호': 36635522,
  '순위': '3',
  '앨범': 'I feel',
  '곡명': '퀸카 (Queencard)',
  '가수': '(여자)아이들',
  '커버이미지_주소': 'https://cdnimg.melon.co.kr/cm2/album/images/112/40/232/11240232_20230509151820_500.jpg/melon/resize/120/quality/80/optimize',
  '좋아요': 42529},
 {'곡일련번호': 36430773,
  '순위': '4',
  '앨범': "NewJeans 2nd EP 'Get Up'",
  '곡명': 'ETA',
  '가수': 'NewJeans',
  '커버이미지_주소': 'https://cdnimg.melon.co.kr/cm2/album/images/112/81/456/11281

In [335]:
import pandas as pd   # 결과

df = 멜론차트_가져오기()

print(df.shape)
df.head()

(100, 7)


,곡일련번호,순위,앨범,곡명,가수,커버이미지_주소,좋아요
0,36599950,1,NewJeans 2nd EP 'Get Up',Super Shy,NewJeans,https://cdnimg.melon.co.kr/cm2/album/images/11...,102366
1,36617841,2,Seven (feat. Latto) - Clean Ver.,Seven (feat. Latto) - Clean Ver.,정국,https://cdnimg.melon.co.kr/cm2/album/images/11...,106437
2,36635522,3,I feel,퀸카 (Queencard),(여자)아이들,https://cdnimg.melon.co.kr/cm2/album/images/11...,42565
3,36430773,4,NewJeans 2nd EP 'Get Up',ETA,NewJeans,https://cdnimg.melon.co.kr/cm2/album/images/11...,122512
4,36382580,5,1집 Alone,헤어지자 말해요,박재정,https://cdnimg.melon.co.kr/cm2/album/images/11...,88725


In [286]:
import re
    
soup = BeautifulSoup(html, "html.parser")

# Find the elements with the specified class 'service_list_song type02 d_song_list'
song_list_items = soup.select("div", class_="service_list_song type02 d_song_list")
#len(song_list_items)
song_title = []
song_title_list = []
song_rank = []
song_rank_list = []
album_list = []
singer_list = []

for song_list_item in song_list_items:
    # Find the 'wrap_song_info' class within 'song_list_item'
    wrap_song_info = song_list_item.find("div", class_="wrap_song_info")
    
    rank_info = song_list_item.find("span", class_="rank")
    
    url_info = song_list_item.find("div", class_="wrap")
    if rank_info:
        song_rank.append(rank_info.text)
        
    if wrap_song_info:
        
        ellipsis_rank01 = wrap_song_info.find("div", class_="ellipsis rank01")
        ellipsis_rank02 = wrap_song_info.find("div", class_="ellipsis rank02")
        ellipsis_rank03 = wrap_song_info.find("div", class_="ellipsis rank03")
        
        if ellipsis_rank01:
            song_title.append(ellipsis_rank01.text.split('\n')[1])
            song_url = ellipsis_rank01.find("a")["href"]
        
        if ellipsis_rank02:
            singer_name = ellipsis_rank02.find("span").text.strip()
            singer_list.append(singer_name)
        if ellipsis_rank03:
            album_list.append(ellipsis_rank03.text.split('\n')[1])   
   
        wrap_song_info = ""

song_title_list = list(dict.fromkeys(song_title))
singer_list = singer_list[6:]

for i in range(len(song_rank)):
    if song_rank[i] != "순위":
        song_rank_list.append(song_rank[i])
        
        



In [287]:
j_url = "https://www.melon.com/commonlike/getSongLike.json?contsIds=36599950%2C36617841%2C36635522%2C36430773%2C36382580%2C36356993%2C36411344%2C36490426%2C36416114%2C36599949%2C34061322%2C36411342%2C36318125%2C35454426%2C36502910%2C34908740%2C35931532%2C35008524%2C35945927%2C36632907%2C36546894%2C35008525%2C32508053%2C35834583%2C35834584%2C36391236%2C35008527%2C36581408%2C36546895%2C35008534%2C35985167%2C35008528%2C35008526%2C35454425%2C35008531%2C32872978%2C35008532%2C36331121%2C33507137%2C36313117%2C35008530%2C34819473%2C35546497%2C35640751%2C30244931%2C36516320%2C8037843%2C36635524%2C35849863%2C36351997%2C36206259%2C36546900%2C33666269%2C34847378%2C36110996%2C36546908%2C36180700%2C36601157%2C36105548%2C36334401%2C36546897%2C36404853%2C35729104%2C36441862%2C36241824%2C36001290%2C36430774%2C36546902%2C36604256%2C34657844%2C4446485%2C36546899%2C36546903%2C36518341%2C36546905%2C36635523%2C36546906%2C36440243%2C35299693%2C33658563%2C36428682%2C36365073%2C34864406%2C34431086%2C35667692%2C36394038%2C34754292%2C36322781%2C36480197%2C35535827%2C35252996%2C35730562%2C36637828%2C36500102%2C36616378%2C33496587%2C36359275%2C36416116%2C35643794%2C36595401"
response = requests.get(j_url, headers={'User-Agent': 'Mozilla/5.0'})
data = response.json()
like_check = []
id_check = []
# 'CONTSID' 키가 있는 데이터만 추출
contsid_data = [item for item in data['contsLike'] if 'SUMMCNT' in item]

for i in range(len(contsid_data)):
    like_check.append(contsid_data[i]['SUMMCNT'])
    id_check.append(contsid_data[i]['CONTSID'])
#print(contsid_data[0]['CONTSID'])
#contsid_data[0]

In [288]:
url = "https://www.melon.com/chart/index.htm"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Find the 'tbody' tag within the HTML
tbody_tag = soup.find("tbody")

img_tags = tbody_tag.find_all("img")

url_list = [img.get("src") for img in img_tags]

url_list



['https://cdnimg.melon.co.kr/cm2/album/images/112/81/456/11281456_20230706180841_500.jpg/melon/resize/120/quality/80/optimize',
 'https://cdnimg.melon.co.kr/cm2/album/images/112/86/070/11286070_20230713181059_500.jpg/melon/resize/120/quality/80/optimize',
 'https://cdnimg.melon.co.kr/cm2/album/images/112/40/232/11240232_20230509151820_500.jpg/melon/resize/120/quality/80/optimize',
 'https://cdnimg.melon.co.kr/cm2/album/images/112/81/456/11281456_20230706180841_500.jpg/melon/resize/120/quality/80/optimize',
 'https://cdnimg.melon.co.kr/cm2/album/images/112/27/533/11227533_20230419162238_500.jpg/melon/resize/120/quality/80/optimize',
 'https://cdnimg.melon.co.kr/cm2/album/images/112/11/297/11211297_20230410151046_500.jpg/melon/resize/120/quality/80/optimize',
 'https://cdnimg.melon.co.kr/cm2/album/images/112/34/678/11234678_20230502162327_500.jpg/melon/resize/120/quality/80/optimize',
 'https://cdnimg.melon.co.kr/cm2/album/images/112/36/264/11236264_20230508184331_500.jpg/melon/resize/12

In [295]:
melon_list = []

for i in range(len(id_check)):
    id_res = id_check[i]
    rank_res = song_rank_list[i]
    album_res = album_list[i] 
    title_res = song_title_list[i]
    name_res = singer_list[i]
    url_res = url_list[i]
    like_res = like_check[i]
    song_info = {
            "곡일련번호": id_res,
            "순위": rank_res,
            "앨범": album_res,
            "곡명": title_res,
            "가수": name_res,
            "커버이미지_주소" : url_res,
            "좋아요" : like_res,
        }
    melon_list.append(song_info)